# Introduction to prompt engineering

*Prompt* เป็นการป้อนคำสั่งให้กับ large language model (LLM) เช่น GPT, Gemini, หรือ Llama เพื่อให้ได้มาซึ่งผลลัพธ์ที่ต้องการ ซึ่งต่อมาได้มีการพัฒนาเทคนิคต่างๆ ในการเขียน prompt ที่ทำให้ผลลัพธ์ที่ได้มีคุณภาพมากขึ้น ซึ่งเราเรียกเทคนิคเหล่านั้นว่า *prompt engineering*

ใน Notebook นี้เราจะยกตัวอย่างการเขียน prompt และ เปรียบเทียบผลลัพธ์ที่ได้จาก Prompt และ โมเดลที่แตกต่างกัน
โดยใช้ *LlamaIndex* ซึ่งเป็น เป็นเฟรมเวิร์กข้อมูลที่เรียบง่ายและยืดหยุ่นสำหรับเชื่อมต่อแหล่งข้อมูลที่กำหนดเองกับ LLM ที่เราต้องการ 

### Prompting

**เริ่มต้นการใช้งาน LlamaIndex**

In [ ]:
# Import Libraries ที่จำเป็น
import os
from PIL import Image
from llama_index.llms.openai import OpenAI
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.core.schema import ImageDocument


os.environ["OPENAI_API_KEY"] = "..."# ใส่ OpenAI API key ที่นี้

**ใช้งาน GPT-4o ด้วย LlamaIndex**

In [ ]:
openai_model = OpenAI(model="gpt-4o")
response = openai_model.complete("What is the meaning of life?")

In [ ]:
print(response)

### Prompt Engineering

Prompt ที่ดีควรจะมีส่วนประกอบดังนี้
- Instruction : งานหรือคำสั่งที่ต้องการให้โมเดลทำ
- Context : ข้อมูลภายนอกหรือบริบทเพิ่มเติมที่ช่วยให้โมเดลตอบได้ดีขึ้น
- Input ข้อมูลหรือคำถามที่เราสนใจหาคำตอบ
- Output Indicator : ประเภทหรือรูปแบบของผลลัพธ์ที่ต้องการ เช่น Tone, Length, Style

ref: https://www.promptingguide.ai/introduction/elements

**Basic Prompt Engineering**

In [ ]:
response = openai_model.complete("Could you list down 5 stunning campaings ideas for my new product launch?")
print(response)

In [ ]:
campaign_prompt = """
In next year, we are going to launch a new food product which is a healthy snack bar.
Could you list down 5 stunning campaings ideas for my new product launch?
The campaign ideas should be in the following format:
# Campaign Name 1 
## Campaign Name
- Campaign Objective
- Campaign Target Audience
- Indicators of Success

# Campaign Name 2
and so on...

The audience for the campaign are Thai Gen Z so the campaign headline should contain some trendy words and in Thai language.
"""
campaign_lists = openai_model.complete(campaign_prompt)
print(campaign_lists)

#### Few-shot Prompting

เป็นการยกตัวอย่างเพียงเล็กน้อยเพื่อให้โมเดลสามารถตอบคำถามได้ตรงตามความต้องการ\
ref: https://www.promptingguide.ai/techniques/fewshot

In [ ]:
baseline = """
Could you list down 5 catchphrases of Thai convenience store 7-Eleven
The catchphrases should be catchy and trendy and must be in Thai language.
"""
baseline_resp = openai_model.complete(baseline)
print(baseline_resp)

In [ ]:
few_shot_prompt = """
Could you list down 5 catchphrases of Thai convenience store 7-Eleven
The catchphrases should be catchy and trendy and must be in Thai language.
here are some examples:
- ไข่สด CP : ไข่สดซีพี เติมสิ่งดีๆ ให้กับชีวิต
- CP : สด สะอาด ปลอดภัย มั่นใจทุกวัน
- CP Fresh Mart : ทุกมื้อดีๆ ที่ซีพีเฟรชมาร์ท
"""
few_shot_resp = openai_model.complete(few_shot_prompt)
print(few_shot_resp)

#### Chain-of-Thought (CoT) Prompting

เป็นการบอกให้โมเดลใช้เหตุผลในการวิเคราะห์คำถาม หรือ คำสั่งก่อนที่จะตอบคำถาม CoT เป็นเทคนิคที่นิยมในการทำให้โมเดลสามารถแก้ปัญหาที่ซับซ้อนได้ดีขึ้น โดยแยกได้ 2 ประเภท คือ
- *Chain-of-Thought (CoT) Prompting* ที่จะเป็นการใส่ลำดับความคิดเข้าไปใน prompt
- *Zero-shot Chain-of-Thought (CoT) Prompting* เป็นการสั่งให้โมเดลคิดอย่างเป็นลำดับเพื่อให้ได้ผลลัพธ์ที่ต้องการ

In [ ]:
the_hardest_problem_for_llm = "อยากให้เมื่อวานเป็นวันพรุ่งนี้จัง วันนี้จะได้เป็นวันศุกร์ วันนี้เป็นวันอะไร?"
baseline_resp = openai_model.complete(the_hardest_problem_for_llm)
print(baseline_resp)

In [ ]:
cot = """
อยากให้เมื่อวานเป็นวันพรุ่งนี้จัง วันนี้จะได้เป็นวันศุกร์ วันนี้เป็นวันอะไร

Let's think step by step:
พรุ่งนี้ ในโลกจริง คือ เมื่อวาน ในโลกสมมติ
เราอยากให้ วันนี้ในโลกสมมติ จะเป็นวันศุกร์ ดังนั้นเมื่อวานในโลกสมมติเป็นวันพฤหัสบดี
ดังนั้น วันนี้ในโลกจริง คือ วันอะไร?
"""
cot_resp = openai_model.complete(cot)
print(cot_resp)

In [ ]:
cot_zero_shot = """
อยากให้เมื่อวานเป็นวันพรุ่งนี้จัง วันนี้จะได้เป็นวันศุกร์ 
วันนี้เป็นวันอะไร
Let's think step by step.
"""
cot_zero_shot_resp = openai_model.complete(cot_zero_shot)
print(cot_zero_shot_resp)

## **Multimodal with ChatGPT & LlmaIndex**

In [ ]:
# Load the image from a path
image_path = '../assets/seki_example.jpg'
image = Image.open(image_path)
display(image)

In [ ]:
openai_mm_llm = OpenAIMultiModal(model="gpt-4o")

In [ ]:
question = """
Perform optical character information and export in the following JSON format.

{
  tax_id: str,
  pos_id: str,
  tel_number: str,
  date: str,
  recepit_number: str,
  items: list[name, float],
  price_before_vat: float,
  total_price: float,
  vat_7_percent: float,
  earn_point: float
}
"""
image_path = '../assets/seki_example.jpg'
recipt = [ImageDocument(image_path=image_path)]
answer = openai_mm_llm.complete(
  prompt=question,
  image_documents=recipt
)
print("Answer:", answer)

## **Explain Nong Moo Deng**

ลองใส่ภาพน้องหมูเด้งแล้วทดลองใช้ ChatGPT อธิบายว่าทำไมน้องถึงเป็น Viral ในโลกออนไลน์

In [ ]:
question = """\
You are a helpful assistant designed to interpret the image and see why the given image gets viral online. You should provide an answer in Thai.
Give an image, please explain why the given image of 'หมูเด้ง', a baby hippo, is viral online?
"""

In [ ]:
image_path = "../assets/moodeng.jpg"
moo_deng = [ImageDocument(image_path=image_path)]
answer = openai_mm_llm.complete(
  prompt=question,
  image_documents=moo_deng
)
print("Answer:", answer)

**LLM อื่นๆที่น่าสนใจ**

- SeaLLM (Alibaba DAMO) https://huggingface.co/spaces/SeaLLMs/SeaLLM-Chat-13b
- Vertex AI (Google) https://cloud.google.com/vertex-ai?hl=en
- Typhoon (SCBX) https://opentyphoon.ai/